# 🎯 ACT Fuzzing Trace Analysis

Interactive visual analysis of ACTFuzzer execution traces.

**Quick Start:**
1. Update `trace_file` path below
2. Run all cells (Cell > Run All)
3. Explore visualizations and use interactive widgets

**Requirements:**
- matplotlib, seaborn (visualization)
- ipywidgets (interactive explorer)
- pandas (data analysis)

---

In [ ]:
# Cell 1: Load and Overview
# ============================================================================

from pathlib import Path
import sys

# Add ACT to path (adjust if needed)
sys.path.insert(0, str(Path.cwd().parent))

from act.pipeline.fuzzing.trace_reader import TraceAnalyzer

# Configure matplotlib for inline display
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Set visual style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# 👇 EDIT THIS: Point to your trace file
# ============================================================================
trace_file = Path("../act/pipeline/log/test_tracing/level_1/traces.json")

# Alternative paths:
# trace_file = Path("../act/pipeline/log/fuzzing_results/traces.json")
# trace_file = Path("../act/pipeline/log/test_tracing/level_2/traces.json")

# ============================================================================
# Load traces and display overview
# ============================================================================

analyzer = TraceAnalyzer(trace_file)
analyzer.show_overview()

In [ ]:
# Cell 2: Visual Overview (Coverage + Strategies)
# ============================================================================
# Displays 2×2 grid of key plots:
#   - Coverage over time (line chart)
#   - Strategy distribution (pie chart)
#   - Strategy effectiveness (box plot)
#   - Violations timeline (scatter plot)
# ============================================================================

analyzer.plot_overview()

In [ ]:
# Cell 3: Interactive Trace Explorer
# ============================================================================
# Use dropdown widget to explore individual traces:
#   - Select trace by iteration
#   - View detailed trace information
#   - Visualize input tensors (before/after/diff heatmaps)
#   - Export selected trace
#
# Requires: pip install ipywidgets
# ============================================================================

analyzer.interactive_explorer()

In [ ]:
# Cell 4: Custom Analysis & Export
# ============================================================================
# This cell is your playground for custom analysis!
#
# Available objects:
#   - analyzer: TraceAnalyzer instance
#   - analyzer.traces: Raw list of trace dicts
#   - df: Pandas DataFrame (uncomment below to create)
#
# Useful methods:
#   - analyzer.show_trace_detail(idx)
#   - analyzer.plot_input_comparison(idx)
#   - analyzer.export_trace(idx, "trace.pt")
#   - analyzer.export_summary("summary.csv")
# ============================================================================

# Uncomment to create DataFrame for analysis
# df = analyzer.to_dataframe()
# display(df.head())

# ============================================================================
# Example 1: Find traces with highest coverage gain
# ============================================================================
print("📊 Top 5 Coverage Gains:\n")
df = analyzer.to_dataframe()
top_traces = df.nlargest(5, 'coverage_delta')[['trace_idx', 'iteration', 'strategy', 'coverage', 'coverage_delta']]
display(top_traces)

# ============================================================================
# Example 2: Analyze a specific trace in detail
# ============================================================================
trace_idx = 0  # Change this to analyze different trace
print(f"\n\n🔍 Detailed Analysis of Trace #{trace_idx}:")
analyzer.show_trace_detail(trace_idx)

# Visualize input if available
if 'input_before' in analyzer.traces[trace_idx]:
    print("\n📊 Input Visualization:")
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    analyzer.plot_input_comparison(trace_idx, axes=axes)
    plt.suptitle(f'Trace #{trace_idx} - Input Tensors', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# ============================================================================
# Example 3: Filter and analyze violations
# ============================================================================
print("\n\n⚠️  Violation Analysis:")
violations = df[df['violation'] == True]

if len(violations) > 0:
    print(f"Found {len(violations)} violations:\n")
    display(violations[['trace_idx', 'iteration', 'strategy', 'coverage']])
    
    # Plot violation strategies
    print("\n📊 Violation Strategy Distribution:")
    strategy_counts = violations['strategy'].value_counts()
    fig, ax = plt.subplots(figsize=(8, 5))
    strategy_counts.plot(kind='bar', ax=ax, color='red', alpha=0.7)
    ax.set_title('Mutations That Found Violations', fontsize=12, fontweight='bold')
    ax.set_xlabel('Strategy')
    ax.set_ylabel('Count')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No violations found in traces")

# ============================================================================
# Example 4: Strategy comparison
# ============================================================================
print("\n\n🔄 Strategy Comparison:")
strategy_stats = df.groupby('strategy').agg({
    'coverage_delta': ['mean', 'std', 'max'],
    'trace_idx': 'count'
}).round(4)
strategy_stats.columns = ['Avg Gain', 'Std Dev', 'Max Gain', 'Count']
display(strategy_stats.sort_values('Avg Gain', ascending=False))

# ============================================================================
# Export Options (uncomment to use)
# ============================================================================

# Export summary to CSV
# analyzer.export_summary("trace_summary.csv")

# Export specific trace
# analyzer.export_trace(trace_idx, f"trace_{trace_idx}.pt")

# Export violations only
# for idx in violations['trace_idx'].values:
#     analyzer.export_trace(idx, f"violation_{idx}.pt")

print("\n✨ Analysis complete!")